In [1]:
import geopandas as gpd

gdf = gpd.read_file("geojson/ACM_building.geojson") #coords in EPSG:4326 (long/lat)
gdf_proj = gdf.to_crs(epsg=3857) #convert geodataframe coord from long/lat to meters

In [61]:
ACM = gdf_proj.geometry 

ACM_edited = ACM.simplify_coverage(2.9)  # 100 meters tolerance


In [62]:
print(ACM_edited.crs)
ACM_edited_4326 = ACM_edited.to_crs("EPSG:4326")
print(ACM_edited_4326.crs)

EPSG:3857
EPSG:4326


In [63]:
def pt(v):
    print(type(v))
    
def count_coords(geom):
    if geom.geom_type == 'Polygon':
        return len(geom.exterior.coords)
    elif geom.geom_type == 'MultiPolygon':
        return sum(len(poly.exterior.coords) for poly in geom.geoms)
    else:
        return None

num_points_before = ACM.apply(count_coords)
num_points_after  = ACM_edited.apply(count_coords)

print("Before simplification:\n", num_points_before)
print("\nAfter simplification:\n", num_points_after)

Before simplification:
 0    105
Name: geometry, dtype: int64

After simplification:
 0    52
dtype: int64


In [64]:
gdf = gpd.GeoDataFrame(geometry=ACM_edited_4326)
gdf.to_file("output.geojson", driver="GeoJSON")

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, LinearRing

# ----- Step 1: Load your GeoJSON -----
gdf = gpd.read_file("geojson/ACM_building.geojson")

geom_series = gdf.geometry

# ----- Step 2: Chaikin smoothing function -----
def chaikin_smooth(coords, iterations=2):
    """
    coords: list of (x, y) tuples
    iterations: number of smoothing passes
    Returns new list of smoothed coordinates
    """
    for _ in range(iterations):
        new_coords = []
        for i in range(len(coords)-1):
            p0, p1 = coords[i], coords[i+1]
            Q = (0.75*p0[0] + 0.25*p1[0], 0.75*p0[1] + 0.25*p1[1])
            R = (0.25*p0[0] + 0.75*p1[0], 0.25*p0[1] + 0.75*p1[1])
            new_coords += [Q, R]
        # Close the ring
        new_coords.append(new_coords[0])
        coords = new_coords
    return coords

# ----- Step 3: Apply smoothing to each geometry -----
def smooth_polygon(geom, iterations=2, simplify_tol=None):
    if geom.geom_type == 'Polygon':
        # Simplify first (optional)
        if simplify_tol:
            geom = geom.simplify(simplify_tol, preserve_topology=True)
        # Apply Chaikin smoothing to exterior
        new_exterior = LinearRing(chaikin_smooth(list(geom.exterior.coords), iterations))
        # Smooth interiors (holes) if any
        new_interiors = [LinearRing(chaikin_smooth(list(ring.coords), iterations)) for ring in geom.interiors]
        return Polygon(new_exterior, new_interiors)
    elif geom.geom_type == 'MultiPolygon':
        return MultiPolygon([smooth_polygon(p, iterations, simplify_tol) for p in geom.geoms])
    else:
        return geom

# Example: 2 Chaikin iterations, optional 3-meter simplification
geom_series_proj = geom_series.to_crs(epsg=3857)
smoothed_series = geom_series_proj.apply(lambda g: smooth_polygon(g, iterations=1, simplify_tol=3))
smoothed_series_4326 = smoothed_series.to_crs(epsg=4326)
# ----- Step 4: Save as GeoJSON -----
gdf_smoothed = gpd.GeoDataFrame(geometry=smoothed_series_4326, crs=gdf.crs)
gdf_smoothed.to_file("smoothed_output.geojson", driver="GeoJSON")


In [76]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, LinearRing

# ----- Step 1: Load original GeoJSON -----
gdf = gpd.read_file("geojson/ACM_building.geojson")
geom_series = gdf.geometry

# ----- Step 2: Snap coordinates to grid -----
def snap_coords(coords, grid_size=3):
    """
    Snap coordinates to a regular grid to reduce zig-zags.
    grid_size in CRS units (meters if projected)
    """
    return [(round(x/grid_size)*grid_size, round(y/grid_size)*grid_size) for x, y in coords]

# ----- Step 3: Convert a polygon to boxy -----
def make_boxy_polygon(geom, grid_size=3):
    if geom.geom_type == 'Polygon':
        new_exterior = LinearRing(snap_coords(list(geom.exterior.coords), grid_size))
        new_interiors = [LinearRing(snap_coords(ring.coords, grid_size)) for ring in geom.interiors]
        return Polygon(new_exterior, new_interiors)
    elif geom.geom_type == 'MultiPolygon':
        return MultiPolygon([make_boxy_polygon(p, grid_size) for p in geom.geoms])
    else:
        return geom

# ----- Step 4: Reproject to meters for proper snapping -----
geom_proj = geom_series.to_crs(epsg=3857)  # EPSG:3857 uses meters

# ----- Step 5: Apply boxy transformation -----
geom_boxy = geom_proj.apply(lambda g: make_boxy_polygon(g, grid_size=5))

# ----- Step 6: Convert back to original CRS (4326) -----
geom_boxy_4326 = geom_boxy.to_crs(epsg=4326)

# ----- Step 7: Save as GeoJSON -----
gpd.GeoDataFrame(geometry=geom_boxy_4326, crs="EPSG:4326").to_file("boxy_output.geojson", driver="GeoJSON")


In [77]:
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon, LinearRing
import math

# ----- Step 1: Load your GeoJSON -----
gdf = gpd.read_file("geojson/ACM_building.geojson")
geom_series = gdf.geometry

# ----- Step 2: Optional function to remove collinear points -----
def remove_collinear(coords, tol=1e-6):
    """
    Removes points that are on a straight line.
    tol: tolerance for floating point errors
    """
    if len(coords) <= 2:
        return coords

    new_coords = [coords[0]]
    for i in range(1, len(coords)-1):
        x0, y0 = new_coords[-1]
        x1, y1 = coords[i]
        x2, y2 = coords[i+1]

        # Compute area of triangle (cross product)
        area = abs((x1 - x0)*(y2 - y0) - (x2 - x0)*(y1 - y0))
        if area > tol:  # keep point if not collinear
            new_coords.append(coords[i])
    new_coords.append(coords[-1])
    return new_coords

# ----- Step 3: Apply simplification and collinear point removal -----
def make_boxy_polygon(geom, tolerance=5):
    """
    Simplifies a polygon to straight-line edges while minimizing vertices.
    tolerance: simplification tolerance in CRS units
    """
    # Simplify using Douglas-Peucker
    geom_simpl = geom.simplify(tolerance, preserve_topology=True)

    # Remove redundant collinear points
    if geom_simpl.geom_type == 'Polygon':
        new_exterior = LinearRing(remove_collinear(list(geom_simpl.exterior.coords)))
        new_interiors = [LinearRing(remove_collinear(list(ring.coords))) for ring in geom_simpl.interiors]
        return Polygon(new_exterior, new_interiors)
    elif geom_simpl.geom_type == 'MultiPolygon':
        return MultiPolygon([make_boxy_polygon(p, tolerance) for p in geom_simpl.geoms])
    else:
        return geom_simpl

# ----- Step 4: Reproject to meters for meaningful tolerance -----
geom_proj = geom_series.to_crs(epsg=3857)

# ----- Step 5: Apply boxy transformation -----
geom_boxy = geom_proj.apply(lambda g: make_boxy_polygon(g, tolerance=5))

# ----- Step 6: Convert back to original CRS (4326) -----
geom_boxy_4326 = geom_boxy.to_crs(epsg=4326)

# ----- Step 7: Save as GeoJSON -----
gpd.GeoDataFrame(geometry=geom_boxy_4326, crs="EPSG:4326").to_file("boxy_output.geojson", driver="GeoJSON")
